In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import random as rand
import itertools
from IPython.display import Audio
from IPython.display import HTML
from IPython.display import Video

b = 100 #size of box
# Setup the figure and axes
fig, ax = plt.subplots(figsize=(8,8))
# Adjust axes limits
ax.set(xlim=(0, b), ylim=(0, b), xlabel='x (m)', ylabel='y (m)',
       title='2D Air Simulation for 100 Molecules')

mN2 = 0.028 #mass of N2 molecule in kg
mO2 = 0.032 #mass of O2 molecule in kg
mAr = 0.040 #mass of Ar molecule in kg
mCO2 = 0.044 #mass of CO2 molecule in kg
rN2 = 3 #radius of N2
rO2 = 2.92 #radius of O2
rAr = 3 #radius of Ar arbitrary
rCO2 = 3 #radius of CO2 arbitrary

T = 10.0 #simulation time in seconds
dt = 0.02 # setting a timestep to be 20 ms given in units of s
N = int(T / dt) #total time steps
M = 10 #number of molecules in our system

p = np.zeros((M, N+1, 2)) #creating array of M molecules, N+1 frames, and 2 dimensions for position
v = np.zeros((M, N+1, 2)) #creating array of M molecules, N+1 frames, and 2 dimensions for velocity

constraint = []
for j in range(M):
    coord = []
    x = rand.randint(1,99)
    y = rand.randint(1,99)
    coord.append(x), coord.append(y)
    
    if coord not in constraint:
        p[j][0] = np.array([x, y]) #Setting initial condition
        constraint.append([x, y])
        
    else:
        M += 1
        

lst=[]
for i in range(M):
    
    theta = np.random.uniform(0,2)#random angle for velocity between 0 and 2 radians
    lottery = rand.random()
    m = []
    
    if 0 <= lottery <= 0.7808:
        
        m.append("Nitrogen"), m.append(mN2), m.append(rN2), m.append("c")
        
        v[i][0] = np.array([15*np.cos(theta), 15*np.sin(theta)]) 
        #uses RMS velocity in m/s of N2 at STP
        
    elif 0.7808 < lottery <= (0.7808+0.2095):
        
        m.append("Oxygen"), m.append(mO2), m.append(rO2), m.append("m")
        
        v[i][0] = np.array([15*np.cos(theta), 15*np.sin(theta)]) 
        #uses RMS velocity in m/s of O2 at STP
        
    elif (0.7808+0.2095) < lottery <= (0.7808+0.2095+0.0093):
        
        m.append("Argon"), m.append(mAr), m.append(rAr), m.append("r")
        
        v[i][0] = np.array([15*np.cos(theta), 15*np.sin(theta)]) 
        #uses RMS velocity in m/s of Argon at STP
        
    elif (0.7808+0.2095+0.0093) < lottery <= 1:
        
        m.append("Carbon Dioxide"), m.append(mCO2), m.append(rCO2), m.append("g")
        
        v[i][0] = np.array([15*np.cos(theta), 15*np.sin(theta)]) 
        #uses RMS velocity in m/s of CO2 at STP
        
    lst.append(m)
    
#function that computes the final velocity vector for each of the 2
#colliding molecules

def velocity(m1,m2,x1,x2,v1,v2):
    """input the mass, position,and velocity of each molecule in
    the collision to compute the resulting velocities"""
    
    v1f = v1 - (2*m2/(m1+m2))*((x1-x2)*np.inner(v1-v2,x1-x2))/(np.linalg.norm(x1-x2))**2
    v2f = v2 - (2*m1/(m1+m2))*((x2-x1)*np.inner(v2-v1,x2-x1))/(np.linalg.norm(x2-x1))**2
    
    return v1f, v2f

#determines if molecule hits box

def box(p,v,r):
    """input position and velocity of molecule"""
    
    #checks if molecule to the left or right of box
    if p[0]-r<=0 or p[0]+r>=b:
        v[0] *= -1
        
    #checks if molecule is above or below the box
    if p[1]-r<=0 or p[1]+r>=b:
        v[1] *=-1
        
    return v

#determines the position of the molecules at the next step

for n in range(N):
    for k in range(M):
        
        p[k][n+1] = p[k][n] + v[k][n] *dt
        
        #check if molecules collides box borders
        v[k][n+1] = box(p[k][n+1], v[k][n], lst[k][2])
        
        #checks if the molecules collide
        for c in itertools.combinations(range(M),2): # gives pairs of molecules
            
            if np.linalg.norm(p[c[0]][n+1] - p[c[1]][n+1])<= (rN2 +rO2):
                
                v[c[0]][n+1],v[c[1]][n+1]=velocity(lst[c[0]][2],
                                                   lst[c[1]][2],
                                                   p[c[0]][n+1],
                                                   p[c[1]][n+1],
                                                   v[c[0]][n+1],
                                                   v[c[1]][n+1])
                
## drawing the first data point
scat = []

for x in range(M):
    scat= np.append(scat, ax.scatter(p[x][0,0], p[x][0,1], marker='o', 
                                     c = lst[x][3], s=(2*lst[x][2])**2))
    
## animating

def animate(z):
    
    for y in range(M):
        
        scat[y].set_offsets(p[y][z])
        
        
ani = animation.FuncAnimation(fig, func=animate, frames=N)

## this function will create a lot of *.png files in a folder
#'CNtower_frames' and create an HTML page with a simulation
ani.save('10mol.html', writer=animation.HTMLWriter(fps= 1//dt))
plt.close()

wave_audio = np.sin(np.linspace(0, 3000, 20000))
Audio(wave_audio, rate=20000, autoplay=True)
#ani.save('CNtower.mp4', fps= 1//dt)

/tmp/ipykernel_3335150/4157993400.py:94: RuntimeWarning: invalid value encountered in true_divide
  v1f = v1 - (2*m2/(m1+m2))*((x1-x2)*np.inner(v1-v2,x1-x2))/(np.linalg.norm(x1-x2))**2
/tmp/ipykernel_3335150/4157993400.py:95: RuntimeWarning: invalid value encountered in true_divide
  v2f = v2 - (2*m1/(m1+m2))*((x2-x1)*np.inner(v2-v1,x2-x1))/(np.linalg.norm(x2-x1))**2


In [1]:
HTML('10mol.html')

NameError: name 'HTML' is not defined